In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_text.csv to sample_text (1).csv


In [ ]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [ ]:
df


,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


# Step 1 : Create source embeddings for the text column

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
!pip install torch sentence-transformers

In [ ]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [ ]:
vectors.shape

(8, 768)

In [ ]:
dim = vectors.shape[1]
dim

768

Build a FAISS Index for vectors

In [ ]:
import faiss

index = faiss.IndexFlatL2(dim)

Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [ ]:
index.add(vectors)

In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f6e6a3f8b70> >

# Step 4 : Encode search text using same encorder and normalize the output vector

In [ ]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [ ]:
vec

array([[ 1.03882691e-02,  2.78686285e-02, -1.18618710e-02,
         1.81327257e-02,  1.21984119e-03, -1.42995641e-02,
         1.96229368e-02,  2.07198318e-02, -2.23660935e-02,
         4.75626774e-02,  1.77976061e-02, -8.00333917e-03,
         2.53419522e-02,  5.26148640e-02,  8.44942406e-03,
        -1.63944215e-02,  1.02661494e-02, -2.74856109e-02,
         8.37067291e-02, -1.52885355e-02,  1.67682748e-02,
        -3.97142116e-03, -2.74348632e-02,  5.02091758e-02,
        -8.36521294e-03, -4.74075824e-02,  2.36915816e-02,
        -1.01186438e-02, -2.82840263e-02,  7.94265140e-03,
         4.21436168e-02, -4.19375999e-03, -1.91232357e-02,
        -3.12785394e-02,  1.24685050e-06, -1.04428437e-02,
        -2.19971444e-02, -8.66928026e-02, -1.88494497e-03,
        -2.54772231e-02, -9.72972624e-03,  7.93310255e-02,
        -3.55963632e-02, -3.05383728e-04, -1.12392241e-02,
        -3.88932489e-02,  5.49313761e-02,  1.35207981e-01,
        -8.19147453e-02,  1.18785072e-02, -9.01845656e-0

In [ ]:
import numpy as np
vec = np.expand_dims(vec, axis=0)
print(vec.shape)

(1, 768)


In [ ]:
# faiss.normalize_L2(svec)

Step 5: Search for similar vector in the FAISS index created

In [ ]:
distances, I = index.search(vec, k=2)
distances

array([[1.3844836, 1.4039097]], dtype=float32)

In [ ]:
I

array([[3, 2]])

In [ ]:
I.tolist()

[[3, 2]]

In [ ]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [ ]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [ ]:
search_query

'I want to buy a polo t-shirt'